In [1]:
import pandas as pd
import psycopg2
from urllib.parse import urlparse
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os

In [2]:
load_dotenv('.env') # Loading the environmental variable

True

In [3]:
# Retrieving the connection string
conn: str = os.getenv('DATABASE_URL')

In [4]:
p = urlparse(conn)

In [5]:
# Establishing the connection to database
connection = psycopg2.connect(
    database = p.path[1:],
    user = p.username,
    password = p.password,
    host = p.hostname,
    port = p.port
)
print(connection)

<connection object at 0x1212eb370; dsn: 'user=postgres password=xxx dbname=demodb host=localhost port=5432', closed: 0>


In [6]:
crime_df = pd.read_csv('Crime_Reports_2008-2024.csv') # Importing the dataset

/var/folders/h4/xk2r0vk53s1cwnltq6fdv2dm0000gn/T/ipykernel_73751/3008504608.py:1: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  crime_df = pd.read_csv('Crime_Reports_2008-2024.csv') # Importing the dataset


In [7]:
crime_df.sample(5, random_state=10) # Sampling the dataset

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
1012713,20152431611,DISTURBANCE - OTHER,3401,N,08/31/2015 08:34:00 PM,08/31/2015,2034.0,08/31/2015 08:34:00 PM,08/31/2015,2034,...,407.00,N,09/09/2015,NaN,NaN,3122209.0,10107272.0,30.367941,-97.715079,"(30.36794147, -97.71507925)"
686975,20125048892,CRIMINAL MISCHIEF,1400,N,11/01/2012 12:00:00 AM,11/01/2012,0.0,11/01/2012 10:40:00 AM,11/01/2012,1040,...,8.00,N,11/05/2012,NaN,NaN,3128388.0,10070989.0,30.267804,-97.698217,"(30.26780446, -97.6982166)"
1425182,20141241009,FAMILY DISTURBANCE/PARENTAL,3458,N,05/04/2014 02:37:00 PM,05/04/2014,1437.0,05/04/2014 03:50:00 PM,05/04/2014,1550,...,410.00,N,09/22/2014,NaN,NaN,3130089.0,10106542.0,30.365422,-97.690154,"(30.36542247, -97.69015426)"
437314,2011280061,BURGLARY OF VEHICLE,601,N,01/28/2011 12:10:00 AM,01/28/2011,10.0,01/28/2011 12:40:00 AM,01/28/2011,40,...,23.14,N,01/28/2011,23F,Theft,3121692.0,10055203.0,30.224846,-97.720596,"(30.22484581, -97.72059573)"
1622924,20102420235,BURGLARY OF VEHICLE,601,N,08/30/2010 03:40:00 AM,08/30/2010,340.0,08/30/2010 03:47:00 AM,08/30/2010,347,...,24.38,N,08/30/2010,23F,Theft,3091679.0,10031181.0,30.160715,-97.817320,"(30.16071527, -97.81732012)"


In [8]:
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820278 entries, 0 to 1820277
Data columns (total 27 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   Incident Number              int64  
 1   Highest Offense Description  object 
 2   Highest Offense Code         int64  
 3   Family Violence              object 
 4   Occurred Date Time           object 
 5   Occurred Date                object 
 6   Occurred Time                float64
 7   Report Date Time             object 
 8   Report Date                  object 
 9   Report Time                  int64  
 10  Location Type                object 
 11  Address                      object 
 12  Zip Code                     float64
 13  Council District             float64
 14  APD Sector                   object 
 15  APD District                 object 
 16  PRA                          object 
 17  Census Tract                 float64
 18  Clearance Status             object 
 19  

In [9]:
# Creating a list for date and timestamp columns before transforming them
date_time_col = ['Occurred Date Time', 'Report Date Time']
date_col = ['Occurred Date', 'Report Date', 'Clearance Date']

In [11]:
for a in crime_df[date_time_col]: # Converting the timestamp columns
    crime_df[a] = pd.to_datetime(crime_df[a], format = '%m/%d/%Y %I:%M:%S %p') # Converting the timestamp datatype with its original format
    crime_df[a] = pd.to_datetime(crime_df[a].dt.strftime('%Y-%m-%d %H:%M:%S')) # Converting the timestamp column again with YYYY-MM-DD HH24:MI:SS format

for b in crime_df[date_col]:
    crime_df[b] = pd.to_datetime(crime_df[b], format = '%m/%d/%Y') # Date columns will be converted as YYYY-MM-DD

In [12]:
crime_df[date_time_col + date_col].info() # Verifying the datatype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820278 entries, 0 to 1820277
Data columns (total 5 columns):
 #   Column              Dtype         
---  ------              -----         
 0   Occurred Date Time  datetime64[ns]
 1   Report Date Time    datetime64[ns]
 2   Occurred Date       datetime64[ns]
 3   Report Date         datetime64[ns]
 4   Clearance Date      datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 69.4 MB


In [15]:
crime_df.sample(5, random_state=10) # Verifying the same sampled values after transforming the datatypes

,Incident Number,Highest Offense Description,Highest Offense Code,Family Violence,Occurred Date Time,Occurred Date,Occurred Time,Report Date Time,Report Date,Report Time,...,Census Tract,Clearance Status,Clearance Date,UCR Category,Category Description,X-coordinate,Y-coordinate,Latitude,Longitude,Location
1012713,20152431611,DISTURBANCE - OTHER,3401,N,2015-08-31 20:34:00,2015-08-31,2034.0,2015-08-31 20:34:00,2015-08-31,2034,...,407.00,N,2015-09-09,NaN,NaN,3122209.0,10107272.0,30.367941,-97.715079,"(30.36794147, -97.71507925)"
686975,20125048892,CRIMINAL MISCHIEF,1400,N,2012-11-01 00:00:00,2012-11-01,0.0,2012-11-01 10:40:00,2012-11-01,1040,...,8.00,N,2012-11-05,NaN,NaN,3128388.0,10070989.0,30.267804,-97.698217,"(30.26780446, -97.6982166)"
1425182,20141241009,FAMILY DISTURBANCE/PARENTAL,3458,N,2014-05-04 14:37:00,2014-05-04,1437.0,2014-05-04 15:50:00,2014-05-04,1550,...,410.00,N,2014-09-22,NaN,NaN,3130089.0,10106542.0,30.365422,-97.690154,"(30.36542247, -97.69015426)"
437314,2011280061,BURGLARY OF VEHICLE,601,N,2011-01-28 00:10:00,2011-01-28,10.0,2011-01-28 00:40:00,2011-01-28,40,...,23.14,N,2011-01-28,23F,Theft,3121692.0,10055203.0,30.224846,-97.720596,"(30.22484581, -97.72059573)"
1622924,20102420235,BURGLARY OF VEHICLE,601,N,2010-08-30 03:40:00,2010-08-30,340.0,2010-08-30 03:47:00,2010-08-30,347,...,24.38,N,2010-08-30,23F,Theft,3091679.0,10031181.0,30.160715,-97.817320,"(30.16071527, -97.81732012)"


In [16]:
new_col_list = [x.lower().replace(' ', '_').replace('-', '_') for x in list(crime_df.columns)] # Changing the column names
new_col_list

['incident_number',
 'highest_offense_description',
 'highest_offense_code',
 'family_violence',
 'occurred_date_time',
 'occurred_date',
 'occurred_time',
 'report_date_time',
 'report_date',
 'report_time',
 'location_type',
 'address',
 'zip_code',
 'council_district',
 'apd_sector',
 'apd_district',
 'pra',
 'census_tract',
 'clearance_status',
 'clearance_date',
 'ucr_category',
 'category_description',
 'x_coordinate',
 'y_coordinate',
 'latitude',
 'longitude',
 'location']

In [17]:
crime_df = crime_df.set_axis(new_col_list, axis = 1)
crime_df.columns

Index(['incident_number', 'highest_offense_description',
       'highest_offense_code', 'family_violence', 'occurred_date_time',
       'occurred_date', 'occurred_time', 'report_date_time', 'report_date',
       'report_time', 'location_type', 'address', 'zip_code',
       'council_district', 'apd_sector', 'apd_district', 'pra', 'census_tract',
       'clearance_status', 'clearance_date', 'ucr_category',
       'category_description', 'x_coordinate', 'y_coordinate', 'latitude',
       'longitude', 'location'],
      dtype='object')

In [18]:
db = create_engine(conn) # Initiate a sqlalchemy db engine from the connection string
demodb_conn = db.connect()

In [20]:
# create the table and insert the records from the dataframe
crime_df.to_sql('austin_crime', con = demodb_conn, if_exists = 'replace', index = False) 

278